We define a 'BayesNet' object to represent a Bayesian Network

In [1]:
class BayesNet:
    
    ## Object representing the probability tables in a Bayes' Net.
    class ProbabilityTable:
    
        ## variables := a dictionary that maps variable names to a specific column number within the table
        ## table := a dictionary with lists as keys and probabilities as values
            ## The keys (lists) represent specific configurations of truth values for each variable
        def __init__(self, variables, table):
            self.variables = variables
            self.table = table
            

    def __init__(self, variables, probability_tables, graph):
        self.variables = variables
        self.probability_tables = probability_tables
        self.graph = graph
        self.indices = {v: k for k, v in self.variables.items()}
    
    ## Method to compute probabilities using inference by enumeration
    ## X := query variables
    ## e := evidence variables
    def inference_by_enumeration(self, X, e, numAlarms):
        numerator = 1
        denominator = 1
        y = [u for u in tuple(self.variables) if u not in X + e]
        for i in self.probability_tables:
            numerator = pointwise_product(numerator, i)
        for i in y:
            ## Adjust code to handle X case
            if i == 'X':
                #Can handle 2, 3, or 4 alarms currently
                if numAlarms == 3:
                    for w in ['X1', 'X2', 'X3']:
                        numerator = sum(w, numerator)
                elif numAlarms == 2:
                    for w in ['X1', 'X2']:
                        numerator = sum(w, numerator)
                elif numAlarms == 4:
                    for w in ['X1', 'X2', 'X3', 'X4']:
                        numerator = sum(w, numerator)
            else:
                numerator = sum(i, numerator)
        denominator = numerator
        for i in X:
            denominator = sum(i, denominator)
        return pointwise_quotient(numerator, denominator)        
        

We define methods to 'multiply' two tables.

In [2]:
## Method to find the pointwise product of two Tables
## f1,f2 := Tables that should be 'multiplied'
## Returns another 'Factor' representing the pointwise product of f1 and f2
def pointwise_product(f1, f2):
    
    ## Trivial pointwise products in which one or more arguments is 0 or 1
    if f1 == 0 or f2 == 0:
        return 0
    if f1 == 1:
        return f2
    if f2 == 1:
        return f1
    
    ## Find the common variable between the two tables to later 'join' on to create the new table
    intersection = tuple(set(f1.variables) & set(f2.variables))
    
    ## Variables of new table are the union of the two input tables
    f3_variables = tuple(list(f1.variables) + [e for e in f2.variables if e not in intersection])
    
    ## Helper dictionary mapping the column numbers resulting table to variable names
    ## This is mainly for bookkeeping purposes throughout the function
    ## e.g. {1: 'A', 2: 'B'} -> Column 1 are values of A; Column 2 are values of B
    dict3 = {}
    for i in range(len(f3_variables)):
        dict3[i] = f3_variables[i]
    
    ## Begin constructing the resultant table f3
    inv_map = {v: k for k, v in dict3.items()}
    f3 = BayesNet.ProbabilityTable(inv_map, {})
    
    for key1 in f1.table:
        for key2 in f2.table:
            
            ## Consider only keys from the two tables that can be 'joined' by the common variable
            if not intersection or (joinable(key1, f1.variables, key2, f2.variables, intersection)):
                
                ## Begin constructing a key that will index an entry in the resulting table
                row = [None] * len(dict3)
                
                ## Take the truth values of the uncommon variables and use them as part of the new key
                for i in range(len(dict3)):
                    variable = dict3[i]
                    if variable in f1.variables:
                        value = key1[f1.variables[variable]]
                        row[i] = value
                    else:
                        value = key2[f2.variables[variable]]
                        row[i] = value
                        
                ## Compute the value (table 'entry') for the newly constructed key and add it to the new table
                entry = round(f1.table[key1] * f2.table[key2], 12)
                f3.table[tuple(row)] = entry
    return f3

We define a method to 'divide' two tables.

In [3]:
## Method to find the pointwise quotient of two Tables
## f1,f2 := Tables that should be 'divided'
## Returns another 'Factor' representing the pointwise quotient of f1 and f2
## Used to normalize conditional probabilities
def pointwise_quotient(f1, f2):
    
    ## Trivial pointwise quotients
    if f1 == 0 and f2 != 0:
        return 0
    if f2 == 1:
        return f1
    
    ## Find the common variable between the two tables to later 'join' on to create the new table
    intersection = tuple(set(f1.variables) & set(f2.variables))
    
    ## Variables of new table are the union of the two input tables
    f3_variables = tuple(list(f1.variables) + [e for e in f2.variables if e not in intersection])
    
    ## Helper dictionary mapping the column numbers resulting table to variable names
    ## This is mainly for bookkeeping purposes throughout the function
    ## e.g. {1: 'A', 2: 'B'} -> Column 1 are values of A; Column 2 are values of B
    dict3 = {}
    for i in range(len(f3_variables)):
        dict3[i] = f3_variables[i]
    
    ## Begin constructing the resultant table f3
    inv_map = {v: k for k, v in dict3.items()}
    f3 = BayesNet.ProbabilityTable(inv_map, {})
    
    for key1 in f1.table:
        for key2 in f2.table:
            
            ## Consider only keys from the two tables that can be 'joined' by the common variable
            if not intersection or (joinable(key1, f1.variables, key2, f2.variables, intersection)):
                
                ## Begin constructing a key that will index an entry in the resulting table
                row = [None] * len(dict3)
                
                ## Take the truth values of the uncommon variables and use them as part of the new key
                for i in range(len(dict3)):
                    variable = dict3[i]
                    if variable in f1.variables:
                        value = key1[f1.variables[variable]]
                        row[i] = value
                    else:
                        value = key2[f2.variables[variable]]
                        row[i] = value
                        
                ## Compute the value (table 'entry') for the newly constructed key and add it to the new table
                if f2.table[key2] != 0:
                    entry = f1.table[key1] / float(f2.table[key2])
                    f3.table[tuple(row)] = entry
                else:
                    f3.table[tuple(row)] = 0
    return f3

We define a utility method to check if two rows in a tables are 'joinable' (can be multiplied or divided).

In [4]:
## Method to check if two "rows" in a probability table are "joinable"
## k1/k2 := "row" 1/2
## vars1/vars2 := dictionary mapping variable names to columns in the table
## intersection := tuple of common variables to 'join' on
def joinable(k1, vars1, k2, vars2, intersection):
    for var in intersection:
        if k1[vars1[var]] != k2[vars2[var]]:
            return False
    return True   

We define a method to sum a table over a variable

In [5]:
## Method to sum over a variable in a probability table.
## var := variable to sum over
## f1 := probability table
def sum(var, f1):
    
    ## Construct resulting variable list
    f2_variables = {}
    col = 0
    for variable in f1.variables:
        if variable != var:
            f2_variables[variable] = col
            col = col + 1
    
    ## Construct resulting table
    f2_table = {}
    for key1 in f1.table:
        
        ## Construct new key in resulting table
        if len(f1.variables) == 1:
            candidate_new_key = [0,]
        else:
            candidate_new_key = [0,]*(len(f1.variables)-1)
        for col in f1.variables:
            if col != var:
                candidate_new_key[f2_variables[col]] = key1[f1.variables[col]]
        candidate_new_key = tuple(candidate_new_key)
        
        ## If the key already exists, then simply the add its value to it's ongoing total
        if candidate_new_key in f2_table:
            f2_table[candidate_new_key] = round(f2_table[candidate_new_key] + f1.table[key1], 15)

        ## Add key to the table if it doesn't exist
        else:
            f2_table[candidate_new_key] = round(f1.table[key1], 15)
    f2 = BayesNet.ProbabilityTable(f2_variables, f2_table)
    return f2

We construct the probability tables given to us in the question

B = burglary <br />
E = earthquake <br />
A = alarm 0 <br />

X_3Alarms = X for 3 additional alarms <br />
J_3Alarms = J for 3 additional alarms <br />
M_ind_3Alarms = M for ind and 3 additional alarms <br />
M_collab_3Alarms = M for collab and 3 additional alarms <br />

X_4Alarms = X for 4 additional alarms <br />
J_4Alarms = J for 4 additional alarms <br />
M_ind_4Alarms = M for ind and 4 additional alarms <br />
M_collab_4Alarms = M for collab and 4 additional alarms <br />

X_2Alarms = X for 2 additional alarms <br />
J_2Alarms = J for 2 additional alarms <br />
M_ind_2Alarms = M for ind and 2 additional alarms <br />
M_collab_2Alarms = M for collab and 2 additional alarms <br />
<br />
We were told in Moodle forum that John calls .99 when half or more of the alarms go off

In [6]:

OH = BayesNet.ProbabilityTable({'OH': 0},
                                 {
                                     (1): .1,
                                     (2): .1,
                                     (3): .1,
                                     (4): .1,
                                     (5): .1,
                                     (6): .1,
                                     (7): .1,
                                     (8): .1,
                                     (9): .1,
                                     (10): .1                     
            })

#might be smarter to make this a function
OB = BayesNet.ProbabilityTable({'OH': 0, 'OB': 1},
                                 {
                                     (1, 1): .9,
                                     (1, 2): .1,
                                     (1, 3): 0,
                                     (1, 4): 0,
                                     (1, 5): 0,
                                     (1, 6): 0,
                                     (1, 7): 0,
                                     (1, 8): 0,
                                     (1, 9): 0,
                                     (1, 10): 0,
                                     (2, 1): .8,
                                     (2, 2): .1,
                                     (2, 3): .1,
                                     (2, 4): 0,
                                     (2, 5): 0,
                                     (2, 6): 0,
                                     (2, 7): 0,
                                     (2, 8): 0,
                                     (2, 9): 0,
                                     (2, 10): 0,
                                     (3, 1): .7,
                                     (3, 2): .1,
                                     (3, 3): .1,
                                     (3, 4): .1,
                                     (3, 5): 0,
                                     (3, 6): 0,
                                     (3, 7): 0,
                                     (3, 8): 0,
                                     (3, 9): 0,
                                     (3, 10): 0,
                                     (4, 1): .6,
                                     (4, 2): .2,
                                     (4, 3): .1,
                                     (4, 4): .1,
                                     (4, 5): 0,
                                     (4, 6): 0,
                                     (4, 7): 0,
                                     (4, 8): 0,
                                     (4, 9): 0,
                                     (4, 10): 0,
                                     (5, 1): .2,
                                     (5, 2): .3,
                                     (5, 3): .3,
                                     (5, 4): .1,
                                     (5, 5): .1,
                                     (5, 6): 0,
                                     (5, 7): 0,
                                     (5, 8): 0,
                                     (5, 9): 0,
                                     (5, 10): 0,
                                     (6, 1): .1,
                                     (6, 2): .2,
                                     (6, 3): .2,
                                     (6, 4): .3,
                                     (6, 5): .1,
                                     (6, 6): .1,
                                     (6, 7): 0,
                                     (6, 8): 0,
                                     (6, 9): 0,
                                     (6, 10): 0,
                                     (7, 1): .1,
                                     (7, 2): .1,
                                     (7, 3): .1,
                                     (7, 4): .1,
                                     (7, 5): .2,
                                     (7, 6): .2,
                                     (7, 7): .2,
                                     (7, 8): 0,
                                     (7, 9): 0,
                                     (7, 10): 0,
                                     (8, 1): 0,
                                     (8, 2): 0,
                                     (8, 3): .1,
                                     (8, 4): .1,
                                     (8, 5): .2,
                                     (8, 6): .2,
                                     (8, 7): .2,
                                     (8, 8): .1,
                                     (8, 9): .1,
                                     (8, 10): 0,
                                     (9, 1): 0,
                                     (9, 2): 0,
                                     (9, 3): 0,
                                     (9, 4): .1,
                                     (9, 5): .1,
                                     (9, 6): .2,
                                     (9, 7): .1,
                                     (9, 8): .1,
                                     (9, 9): .2,
                                     (9, 10): .2,
                                     (10, 1): 0,
                                     (10, 2): 0,
                                     (10, 3): 0,
                                     (10, 4): 0,
                                     (10, 5): 0,
                                     (10, 6): 0,
                                     (10, 7): 0,
                                     (10, 8): 0,
                                     (10, 9): .1,
                                     (10, 10): .9
                                     
            })

#Need to write a function that determines this probability distribution instead of a dictionary
PG = BayesNet.ProbabilityTable({'PG': 0, 'OB': 1},
                                 {                     
                                     #this table is basically like the inverse of the 'his bet' table
                                     #needs to be defaulted to some sort of basic value--an initial belief / understanding
                                     #also needs to be a function
                                     
                                     (1, 1): .1,
                                     (1, 2): .1,
                                     (1, 3): .1,
                                     (1, 4): .1,
                                     (1, 5): .1,
                                     (1, 6): .1,
                                     (1, 7): .1,
                                     (1, 8): .1,
                                     (1, 9): .1,
                                     (1, 10): .1,
                                     (2, 1): .1,
                                     (2, 2): .1,
                                     (2, 3): .1,
                                     (2, 4): .1,
                                     (2, 5): .1,
                                     (2, 6): .1,
                                     (2, 7): .1,
                                     (2, 8): .1,
                                     (2, 9): .1,
                                     (2, 10): .1,
                                     (3, 1): .1,
                                     (3, 2): .1,
                                     (3, 3): .1,
                                     (3, 4): .1,
                                     (3, 5): .1,
                                     (3, 6): .1,
                                     (3, 7): .1,
                                     (3, 8): .1,
                                     (3, 9): .1,
                                     (3, 10): .1,
                                     (4, 1): .1,
                                     (4, 2): .1,
                                     (4, 3): .1,
                                     (4, 4): .1,
                                     (4, 5): .1,
                                     (4, 6): .1,
                                     (4, 7): .1,
                                     (4, 8): .1,
                                     (4, 9): .1,
                                     (4, 10): .1,
                                     (5, 1): .1,
                                     (5, 2): .1,
                                     (5, 3): .1,
                                     (5, 4): .1,
                                     (5, 5): .1,
                                     (5, 6): .1,
                                     (5, 7): .1,
                                     (5, 8): .1,
                                     (5, 9): .1,
                                     (5, 10): .1,
                                     (6, 1): .1,
                                     (6, 2): .1,
                                     (6, 3): .1,
                                     (6, 4): .1,
                                     (6, 5): .1,
                                     (6, 6): .1,
                                     (6, 7): .1,
                                     (6, 8): .1,
                                     (6, 9): .1,
                                     (6, 10): .1,
                                     (7, 1): .1,
                                     (7, 2): .1,
                                     (7, 3): .1,
                                     (7, 4): .1,
                                     (7, 5): .1,
                                     (7, 6): .1,
                                     (7, 7): .1,
                                     (7, 8): .1,
                                     (7, 9): .1,
                                     (7, 10): .1,
                                     (8, 1): .1,
                                     (8, 2): .1,
                                     (8, 3): .1,
                                     (8, 4): .1,
                                     (8, 5): .1,
                                     (8, 6): .1,
                                     (8, 7): .1,
                                     (8, 8): .1,
                                     (8, 9): .1,
                                     (8, 10): .1,
                                     (9, 1): .1,
                                     (9, 2): .1,
                                     (9, 3): .1,
                                     (9, 4): .1,
                                     (9, 5): .1,
                                     (9, 6): .1,
                                     (9, 7): .1,
                                     (9, 8): .1,
                                     (9, 9): .1,
                                     (9, 10): .1,
                                     (10, 1): .1,
                                     (10, 2): .1,
                                     (10, 3): .1,
                                     (10, 4): .1,
                                     (10, 5): .1,
                                     (10, 6): .1,
                                     (10, 7): .1,
                                     (10, 8): .1,
                                     (10, 9): .1,
                                     (10, 10): .1
            })

PH = BayesNet.ProbabilityTable({'PH': 0},
                                 {
                                     (1): .1,
                                     (2): .1,
                                     (3): .1,
                                     (4): .1,
                                     (5): .1,
                                     (6): .1,
                                     (7): .1,
                                     (8): .1,
                                     (9): .1,
                                     (10): .1                                         
            })

#this needs to likely be some sort of function
#how are these probabilities supposed to be set?
PA = BayesNet.ProbabilityTable({'PA': 0, 'PH': 1, 'PG': 2},
                                 {
                                     ('C', 1, 1): .1,
                                     ('C', 1, 2): .1,
                                     ('C', 1, 3): .1,
                                     ('C', 1, 4): .1,
                                     ('C', 1, 5): .1,
                                     ('C', 1, 6): .1,
                                     ('C', 1, 7): .1,
                                     ('C', 1, 8): .1,
                                     ('C', 1, 9): .1,
                                     ('C', 1, 10): .1,
                                     ('C', 2, 1): .1,
                                     ('C', 2, 2): .1,
                                     ('C', 2, 3): .1,
                                     ('C', 2, 4): .1,
                                     ('C', 2, 5): .1,
                                     ('C', 2, 6): .1,
                                     ('C', 2, 7): .1,
                                     ('C', 2, 8): .1,
                                     ('C', 2, 9): .1,
                                     ('C', 2, 10): .1,
                                     ('C', 3, 1): .1,
                                     ('C', 3, 2): .1,
                                     ('C', 3, 3): .1,
                                     ('C', 3, 4): .1,
                                     ('C', 3, 5): .1,
                                     ('C', 3, 6): .1,
                                     ('C', 3, 7): .1,
                                     ('C', 3, 8): .1,
                                     ('C', 3, 9): .1,
                                     ('C', 3, 10): .1,
                                     ('C', 4, 1): .1,
                                     ('C', 4, 2): .1,
                                     ('C', 4, 3): .1,
                                     ('C', 4, 4): .1,
                                     ('C', 4, 5): .1,
                                     ('C', 4, 6): .1,
                                     ('C', 4, 7): .1,
                                     ('C', 4, 8): .1,
                                     ('C', 4, 9): .1,
                                     ('C', 4, 10): .1,
                                     ('C', 5, 1): .1,
                                     ('C', 5, 2): .1,
                                     ('C', 5, 3): .1,
                                     ('C', 5, 4): .1,
                                     ('C', 5, 5): .1,
                                     ('C', 5, 6): .1,
                                     ('C', 5, 7): .1,
                                     ('C', 5, 8): .1,
                                     ('C', 5, 9): .1,
                                     ('C', 5, 10): .1,
                                     ('C', 6, 1): .1,
                                     ('C', 6, 2): .1,
                                     ('C', 6, 3): .1,
                                     ('C', 6, 4): .1,
                                     ('C', 6, 5): .1,
                                     ('C', 6, 6): .1,
                                     ('C', 6, 7): .1,
                                     ('C', 6, 8): .1,
                                     ('C', 6, 9): .1,
                                     ('C', 6, 10): .1,
                                     ('C', 7, 1): .1,
                                     ('C', 7, 2): .1,
                                     ('C', 7, 3): .1,
                                     ('C', 7, 4): .1,
                                     ('C', 7, 5): .1,
                                     ('C', 7, 6): .1,
                                     ('C', 7, 7): .1,
                                     ('C', 7, 8): .1,
                                     ('C', 7, 9): .1,
                                     ('C', 7, 10): .1,
                                     ('C', 8, 1): .1,
                                     ('C', 8, 2): .1,
                                     ('C', 8, 3): .1,
                                     ('C', 8, 4): .1,
                                     ('C', 8, 5): .1,
                                     ('C', 8, 6): .1,
                                     ('C', 8, 7): .1,
                                     ('C', 8, 8): .1,
                                     ('C', 8, 9): .1,
                                     ('C', 8, 10): .1,
                                     ('C', 9, 1): .1,
                                     ('C', 9, 2): .1,
                                     ('C', 9, 3): .1,
                                     ('C', 9, 4): .1,
                                     ('C', 9, 5): .1,
                                     ('C', 9, 6): .1,
                                     ('C', 9, 7): .1,
                                     ('C', 9, 8): .1,
                                     ('C', 9, 9): .1,
                                     ('C', 9, 10): .1,
                                     ('C', 10, 1): .1,
                                     ('C', 10, 2): .1,
                                     ('C', 10, 3): .1,
                                     ('C', 10, 4): .1,
                                     ('C', 10, 5): .1,
                                     ('C', 10, 6): .1,
                                     ('C', 10, 7): .1,
                                     ('C', 10, 8): .1,
                                     ('C', 10, 9): .1,
                                     ('C', 10, 10): .1,
                                     ('F', 1, 1): .1,
                                     ('F', 1, 2): .1,
                                     ('F', 1, 3): .1,
                                     ('F', 1, 4): .1,
                                     ('F', 1, 5): .1,
                                     ('F', 1, 6): .1,
                                     ('F', 1, 7): .1,
                                     ('F', 1, 8): .1,
                                     ('F', 1, 9): .1,
                                     ('F', 1, 10): .1,
                                     ('F', 2, 1): .1,
                                     ('F', 2, 2): .1,
                                     ('F', 2, 3): .1,
                                     ('F', 2, 4): .1,
                                     ('F', 2, 5): .1,
                                     ('F', 2, 6): .1,
                                     ('F', 2, 7): .1,
                                     ('F', 2, 8): .1,
                                     ('F', 2, 9): .1,
                                     ('F', 2, 10): .1,
                                     ('F', 3, 1): .1,
                                     ('F', 3, 2): .1,
                                     ('F', 3, 3): .1,
                                     ('F', 3, 4): .1,
                                     ('F', 3, 5): .1,
                                     ('F', 3, 6): .1,
                                     ('F', 3, 7): .1,
                                     ('F', 3, 8): .1,
                                     ('F', 3, 9): .1,
                                     ('F', 3, 10): .1,
                                     ('F', 4, 1): .1,
                                     ('F', 4, 2): .1,
                                     ('F', 4, 3): .1,
                                     ('F', 4, 4): .1,
                                     ('F', 4, 5): .1,
                                     ('F', 4, 6): .1,
                                     ('F', 4, 7): .1,
                                     ('F', 4, 8): .1,
                                     ('F', 4, 9): .1,
                                     ('F', 4, 10): .1,
                                     ('F', 5, 1): .1,
                                     ('F', 5, 2): .1,
                                     ('F', 5, 3): .1,
                                     ('F', 5, 4): .1,
                                     ('F', 5, 5): .1,
                                     ('F', 5, 6): .1,
                                     ('F', 5, 7): .1,
                                     ('F', 5, 8): .1,
                                     ('F', 5, 9): .1,
                                     ('F', 5, 10): .1,
                                     ('F', 6, 1): .1,
                                     ('F', 6, 2): .1,
                                     ('F', 6, 3): .1,
                                     ('F', 6, 4): .1,
                                     ('F', 6, 5): .1,
                                     ('F', 6, 6): .1,
                                     ('F', 6, 7): .1,
                                     ('F', 6, 8): .1,
                                     ('F', 6, 9): .1,
                                     ('F', 6, 10): .1,
                                     ('F', 7, 1): .1,
                                     ('F', 7, 2): .1,
                                     ('F', 7, 3): .1,
                                     ('F', 7, 4): .1,
                                     ('F', 7, 5): .1,
                                     ('F', 7, 6): .1,
                                     ('F', 7, 7): .1,
                                     ('F', 7, 8): .1,
                                     ('F', 7, 9): .1,
                                     ('F', 7, 10): .1,
                                     ('F', 8, 1): .1,
                                     ('F', 8, 2): .1,
                                     ('F', 8, 3): .1,
                                     ('F', 8, 4): .1,
                                     ('F', 8, 5): .1,
                                     ('F', 8, 6): .1,
                                     ('F', 8, 7): .1,
                                     ('F', 8, 8): .1,
                                     ('F', 8, 9): .1,
                                     ('F', 8, 10): .1,
                                     ('F', 9, 1): .1,
                                     ('F', 9, 2): .1,
                                     ('F', 9, 3): .1,
                                     ('F', 9, 4): .1,
                                     ('F', 9, 5): .1,
                                     ('F', 9, 6): .1,
                                     ('F', 9, 7): .1,
                                     ('F', 9, 8): .1,
                                     ('F', 9, 9): .1,
                                     ('F', 9, 10): .1,
                                     ('F', 10, 1): .1,
                                     ('F', 10, 2): .1,
                                     ('F', 10, 3): .1,
                                     ('F', 10, 4): .1,
                                     ('F', 10, 5): .1,
                                     ('F', 10, 6): .1,
                                     ('F', 10, 7): .1,
                                     ('F', 10, 8): .1,
                                     ('F', 10, 9): .1,
                                     ('F', 10, 10): .1
            })



To represent the structure of the Bayes Net, we use a adjacency matrix.

In [7]:
graph = [[0 for col in range(5)] for row in range(5)]
graph[0][1] = 1
graph[1][2] = 1
graph[2][4] = 1
graph[3][4] = 1

In [8]:
def parse_results(solution, evidDict):   
    prob = 0
    for j in solution.table:
                key = ['' for z in range(len(solution.variables))]
                for x in solution.variables:
                    key[int(solution.variables[x])] = x
                for n in range(len(key)):
                    if key[n] in evidDict:
                        key[n] = evidDict[key[n]]
                    else:
                        key[n] = True
                key = tuple(key)
                if key == j:
                    prob = solution.table[j]
    return prob

We construct BayesNet Objects corresponding to the different scenarios laid out in the question

In [9]:
bn = BayesNet({'OH': 0, 'OB': 1, 'PG': 2, 'PH': 3, 'PA': 4}, [OH, OB, PG, PH, PG], graph)